# Summary

In [1]:
from abstractor.train import get_training_batch as get_abstractor_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.train import get_training_batch as get_extractor_training_batch
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rl_connection.utils import RLModel
from rl_connection.train import get_training_batch as get_rl_training_batch
from rouge import Rouge

import numpy as np
import torch

## Load data

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
data = load_training_dictionaries()

## Extractor

In [4]:
# Load extractor model:
extractor_model = ExtractorModel(bert_tokenizer, bert_model)
extractor_model_path = "results/models/extractor.pt"
extractor_model.load_state_dict(torch.load(extractor_model_path))

<All keys matched successfully>

In [5]:
documents, extraction_labels = get_extractor_training_batch(data, batch_size=2)

sentence_embeddings, mask = obtain_sentence_embeddings(
    extractor_model.bert_model, 
    extractor_model.bert_tokenizer, 
    documents
)

# Predict probability of extraction per sentence
extraction_probabilities = extractor_model(sentence_embeddings)

In [6]:
n_samples = len(documents)

for sample_idx in range(n_samples):
    n_to_extract = extraction_labels.sum(dim=1)[sample_idx].int() 
    ext_prob = extraction_probabilities[sample_idx] * mask[sample_idx]
    ext_sent_indicies = torch.topk(ext_prob, k=n_to_extract)[1]
    
    targets = np.array(documents[sample_idx])[extraction_labels[sample_idx][:len(documents[sample_idx])].numpy().astype(bool)]
    print("----> TARGET <----")
    for target in targets:
        print(f"{target}\n")
    print()
          
    print("----> PREDICTION <----")
    for x in np.array(documents[sample_idx])[ext_sent_indicies]:
        print(f"{x} \n")
    print("\n\n-------\n\n")

----> TARGET <----
" weasels will go for anything that looks like food -- they 've got a high metabolism and they 've got to eat a lot , " she said . " it does n't surprise me that a weasel took a punt -- i 've seen a photo of a weasel charging a group of sparrows , they 're very hungry animals . "

weasels would not normally target green woodpeckers , pacheco said -- their predators are normally the size of a stoat or larger . but the birds are known to spend a fair amount of time on the ground pulling up worms and hunting insects .

the pluckiness of the weasel spawned a number of parodies on twitter , with manipulated images showing the creature in turn being ridden by russian president vladimir putin , popstar miley cyrus , football star john terry -- and even what appears to be a dog red panda dressed in a darth vader costume . ( update : twitter has now educated us on the difference between a dog and a red panda . sorry , darth ! )


----> PREDICTION <----
the pluckiness of the w

## Abstractor

In [80]:
# Load data:
abstractor_model = AbstractorModelRNN(bert_tokenizer, bert_model)
abstractor_model_path = "results/models/abstractor.pt"
abstractor_model.load_state_dict(torch.load(abstractor_model_path))

<All keys matched successfully>

In [92]:
source_documents, target_summaries = get_abstractor_training_batch(data, 1)

# Obtain embeddings
source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, source_documents, static_embeddings=False
)
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, target_summaries, static_embeddings=True
)

print(source_documents)
print()
print(target_summaries)

[["two american women have reportedly been arrested for carving their initials into a wall with a coin inside rome 's colosseum .", "cambodia 's angkor archeological park experienced its own string of nudity - related incidents this year .", "in february , u.s. tourists and sisters lindsey kate adams and leslie jan adams were deported after allegedly getting caught taking partially nude photos at preah khan temple , one of the sacred sites inside cambodia 's angkor complex ."]]

[['two american women arrested for carving initials into a colosseum wall .', 'meanwhile , egypt investigating russian pornography film reportedly shot at great pyramids .', "cambodia 's angkor archeological park experienced a string of nudity - related incidents this year ."]]


In [82]:
# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=0
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")
    
print("------------")


# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=1
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")

['zu', '##lly', 'bro', '##uss', '##ard', 'decided', 'to', 'give', 'a', 'kidney', 'for', 'give', 'a', 'kidney', 'for', 'give', 'a', 'kidney', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for', 'give', 'for']

['carving', 'initials', 'into', 'a', 'col', '##oss', '##eum', 'wall', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile', '##s', '.', 'meanwhile']

------------
['zu', '##lly', 'bro', '##uss', '##ard', 'decided', 'to', 'give', 'a', 'kidney', 'for', 'a', 'kidney', '.', 'a', 'new', 'computer', 'program', 'helped', 'her', 'donation', 'spur', 'transplant', '##s'

## Reinforcement Learning

In [95]:
rl_model = RLModel(extractor_model, abstractor_model)
rl_model.load_state_dict(torch.load("results/models/rl.pt"))

<All keys matched successfully>

In [101]:
source_documents, target_summaries = get_rl_training_batch(data, batch_size=1)

# Obtain embeddings
source_sentence_embeddings, source_mask = obtain_sentence_embeddings(
    rl_model.extractor_model.bert_model,
    rl_model.extractor_model.bert_tokenizer,
    source_documents
)
stop_action_index = source_sentence_embeddings.shape[1]
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    rl_model.abstractor_model.bert_model,
    rl_model.abstractor_model.bert_tokenizer,
    target_summaries,
    static_embeddings=True
)

In [102]:
# Run trajectory
actions, log_probs, entropys, values = rl_model.sample_actions(source_sentence_embeddings, source_mask)

# Obtain abstracted sentence from abstractor
predicted_tokens, word_probabilities = rl_model.create_abstracted_sentences(
    actions,
    source_documents,
    stop_action_index,
    teacher_forcing_pct=1.0,
    target_summary_embeddings=target_summary_embeddings
)
        

In [103]:
# Look at extractions
for art_idx, doc_sentences in enumerate(actions):
    for sent_idx in doc_sentences[:-1]:
        print(source_documents[art_idx][sent_idx])
        print()
    print("\n\n-------\n\n")

( cnn ) my vote for father of the year goes to curt schilling . the former major league baseball pitcher recently fired off a series of fastballs and mowed down a group of twitter trolls who made the mistake of tweeting vulgar and sexually - explicit comments about schilling 's teenage daughter .

these misogynistic cowards may have thought they could hide in the darkness of anonymity , the sort that many have come to expect from social media sites , where you feel free to be a despicable human being because , you think , no one will ever find out who you really are and hold you accountable for your words .

finally , it 's worth reminding everyone that freedom of expression does not mean freedom from rules , standards , and expectations that should guide your behavior . there are things you do n't say . there are boundaries , ways that we expect you to behave so you do n't terrorize other people or bring shame upon yourself , your friends , and your family . if you do n't have social 

In [104]:
# Look at abstractions
for predicted_abstraction in predicted_tokens:
    solution = list()
    for token in predicted_abstraction:
        solution.append(rl_model.abstractor_model.bert_tokenizer.ids_to_tokens[int(token)])
    print(" ".join(solution))
    print("\n\n")

ruben navarre ##tte ##tte sc ##hill ##ing deserves praise for taking on online hate ##rs for offensive comments about his daughter . navarre his his in his his child , sc in ##ing set a model for parenting and taught us a lesson about social media . [SEP] .





In [105]:
target_summaries

[['ruben navarrette : schilling deserves praise for taking on online haters for offensive comments about his daughter .',
  'navarrette : in protecting his child , schilling set a model for parenting and taught us a lesson about social media .']]